### Example for NaCl cell optimization using ASE-LBFGS and Conquest

In [ ]:
import os

from ase.build import bulk
from ase.calculators.conquest import Conquest
from ase.constraints import UnitCellFilter
from ase.optimize import LBFGS
from ase.io.conquest import Conquest_orthorhombic_check

#### Define Conquest environment

In [ ]:
os.environ['ASE_CONQUEST_COMMAND'] = 'mpirun -np 4 /Users/lioneltruflandier/CONQUEST-release-f-rework-output-ase/src/Conquest'
#os.environ['ASE_CONQUEST_COMMAND'] = 'mpirun -np 4 /Users/lioneltruflandier/CONQUEST-develop/src/Conquest'
os.environ['CQ_PP_PATH'] = '/Users/lioneltruflandier/Conquest-develop-outdated/pseudo-and-pao'
os.environ['CQ_GEN_BASIS_CMD'] = '/Users/lioneltruflandier/CONQUEST-release-develop/tools/BasisGeneration/MakeIonFiles'

#### Directory for storing calculation files

In [ ]:
working_directory = 'cq_example_rocksalt_optcell_ase'

#### Generate rocksalt struture with $a=6.0$ Ang.

In [ ]:
rocksalt = bulk('NaCl', crystalstructure='rocksalt', a=6.0)

# Conquest can only handle orthorhombic cells
rocksalt = Conquest_orthorhombic_check(rocksalt,verbose=True)

#### Setup Conquest atomic basis set

In [ ]:
basis = {'Na' : {'gen_basis'            : True,
                 'basis_size'           : 'medium',
                 'pseudopotential_type' : 'hamann'},
         'Cl' : {'gen_basis'            : False,
                 'basis_size'           : 'medium',
                 'pseudopotential_type' : 'hamann'
                 }}

#basis = {'Na' : {'file' : 'Na_PBE_DZP_CQ.ion'},
#         'Cl' : {'file' : 'Cl_PBE_DZP_CQ.ion'}}
#
#basis = {'Na' : {'gen_basis'            : False,
#                 'basis_size'           : 'medium',
#                 'pseudopotential_type' : 'hamann',
#                 'xc'                   : 'PBE' },
#         'Cl' : {'file' : 'Cl_PBE_DZP_CQ.ion', 'xc' : 'PBE'}
#         }

#### Setup calculation using Conquest as calculator

In [ ]:
conquest_flags = {'IO.WriteOutToASEFile': True}

calc = Conquest(directory=working_directory,
                grid_cutoff=80,
                xc='PBE',
                self_consistent=True,
                basis=basis,
                kpts=[3,3,3],
                nspin=1,
                **conquest_flags)

#### Run calculation

In [ ]:
rocksalt.calc = calc
dft_energy    = rocksalt.get_potential_energy()

In [ ]:
ucf = UnitCellFilter(rocksalt)
opt = LBFGS(ucf)

In [ ]:
opt.run(fmax=0.005)

#### Compute final lattice constant

In [ ]:
exp_lattice = 5.64
thr_lattice = (8*rocksalt.get_volume()/len(rocksalt))**(1.0/3.0)
print('exp lattice contant = {:8.4f}'.format(exp_lattice))
print('th  lattice contant = {:8.4f}'.format(thr_lattice))